<a href="https://colab.research.google.com/github/kyoungyi/Small-Projects/blob/master/all_county_corn_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! pip install cftime
import xarray as xr
import numpy as np
import pandas as pd
import scipy.io
from itertools import zip_longest


In [3]:
data = scipy.io.loadmat('/content/drive/My Drive/all_crops_LLSupdated07282020_statesAndCounties.mat', squeeze_me=True)
county = data['st']['c']
#a.dtype.names
crop_names = county[0].dtype.names

('name',
 'FIPS',
 'c',
 'corn_yrs',
 'corn_yld',
 'corn_area_yrs',
 'corn_area',
 'soy_yrs',
 'soy_yld',
 'soy_area_yrs',
 'soy_area',
 'wwht_yrs',
 'wwht_yld',
 'wwht_area_yrs',
 'wwht_area')

In [4]:
time = xr.cftime_range(start="1910-01-01", periods=110, freq="1Y", calendar='noleap')
time.year

In [ ]:
for i in range(len(county)): #len(county)
  for j in range(len(county[i])): #len(county[i])
    if len(np.atleast_1d(county[i]['corn_yld_yrs'][j])) != 0 or len(np.atleast_1d(county[i]['corn_area_yrs'][j])) !=0:
      corn_yr = np.atleast_1d(county[i]['corn_yld_yrs'][j]) #.astype('int64')
      corn_yld = np.atleast_1d(county[i]['corn_yld'][j]) #.astype('int64')
      corn_area_yr =np.atleast_1d(county[i]['corn_area_yrs'][j]) #.astype('int64')
      corn_area = np.atleast_1d(county[i]['corn_area'][j]) #.astype('int64')
      temp1  = np.zeros([110])
      temp2 = np.zeros([110])
      index1 = []
      index2 = []
      idx1 = 0
      idx2 = 0
      print(i,j)
      for z,k in zip_longest(corn_yr,corn_area_yr):
        if z != None:
          idx1 = np.where(list(time.year) == z)
          index1.append(idx1)
        if k != None:
          idx2= np.where(list(time.year) == k)
          index2.append(idx2)
      #print(np.ravel(index1), np.ravel(index2))
      if len(corn_yr) !=0:
        temp1[np.ravel(index1)] = corn_yld
      if len(corn_area_yr) !=0:
        temp2[np.ravel(index2)] = corn_area
      #print(temp1[np.ravel(index1)], corn_yld,county[i]["mLat"][j])

      # Saving: change from dict to dataframe
      my_dict = dict(state_name=data['st']['name'][i],state_id=data['st']['FIPS'][i],
                    county_name=county[i]['name'][j],county_id=county[i]['ID'][j],
                    lat=county[i]["mLat"][j],lon=county[i]["mLon"][j],
                    year=time,corn_yld=temp1,corn_area=temp2)
      dataframe_temp = pd.DataFrame.from_dict(my_dict)
      
      if i == 0 and j == 0:
        dataframe = dataframe_temp
      else:
        dataframe = dataframe.append(dataframe_temp,ignore_index=True, sort=False)
    
    else:
      print('no data: ',i,j)



In [ ]:
dataframe

In [ ]:
dataframe.to_csv('/content/drive/My Drive/state_county_corn.csv')